In [1]:
# 🔧 FINE-TUNING CONFIGURATION
# ============================================================================
# 🎯 Optimized for fine-tuning with 20 unfrozen layers
# 🔥 Lower learning rates and more epochs for stable convergence

CONFIG_FT = {
    # === ARCHITECTURE ===
    'IMG_SIZE': 224,                    # ✅ Optimal pour EfficientNet
    'BATCH_SIZE': 16,                   # 🔄 Reduced for fine-tuning stability
    'EMBEDDING_DIM_CUSTOM': 512,        # ✅ Rich embeddings
    'UNFROZEN_LAYERS': 20,              # 🆕 Number of layers to unfreeze
    
    # === TRAINING HYPERPARAMETERS ===
    'FINETUNE_EPOCHS': 25,              # 🆕 More epochs for fine-tuning
    'FINETUNE_LR': 0.00005,             # 🆕 Very low LR for fine-tuning
    'HEAD_LR': 0.001,                   # 🆕 Higher LR for new head layers
    
    # === REGULARIZATION ===
    'DROPOUT_RATE': 0.4,                # 🔄 Higher dropout for fine-tuning
    'WEIGHT_DECAY': 0.0001,             # ✅ L2 regularization
    'LABEL_SMOOTHING': 0.1,             # 🆕 Label smoothing for stability
    
    # === DATA AUGMENTATION ===
    'USE_AUGMENTATION': True,           # ✅ Data augmentation
    'VALIDATION_SPLIT': 0.2,           # ✅ Train/val split
    
    # === TRAINING CONTROL ===
    'PATIENCE': 5,                      # 🆕 Early stopping patience
    'REDUCE_LR_PATIENCE': 3,           # 🆕 LR reduction patience
    'REDUCE_LR_FACTOR': 0.5,           # 🆕 LR reduction factor
    'MIN_LR': 1e-7,                    # 🆕 Minimum learning rate
    
    # === MODEL SAVING ===
    'MODEL_NAME': 'recipe_image_retrieval_model_ft.keras',
    'CHECKPOINT_DIR': './ft/',          # 🆕 Fine-tuning model directory
    
    # === TRIPLET LOSS ===
    'MARGIN': 0.2,                     # ✅ Triplet loss margin
    'HARD_NEGATIVE_MINING': True,      # 🆕 Use hard negative mining
}

# Créer le dossier de sauvegarde
import os
os.makedirs(CONFIG_FT['CHECKPOINT_DIR'], exist_ok=True)

print("🔥 FINE-TUNING CONFIGURATION")
print("=" * 50)
print(f"📐 Image Size: {CONFIG_FT['IMG_SIZE']}x{CONFIG_FT['IMG_SIZE']}")
print(f"📦 Batch Size: {CONFIG_FT['BATCH_SIZE']}")
print(f"🔥 Unfrozen Layers: {CONFIG_FT['UNFROZEN_LAYERS']}")
print(f"🔄 Max Epochs: {CONFIG_FT['FINETUNE_EPOCHS']}")
print(f"📈 Fine-tune LR: {CONFIG_FT['FINETUNE_LR']}")
print(f"🎯 Head LR: {CONFIG_FT['HEAD_LR']}")
print(f"💧 Dropout: {CONFIG_FT['DROPOUT_RATE']}")
print(f"⚖️ Weight Decay: {CONFIG_FT['WEIGHT_DECAY']}")
print(f"⏰ Early Stopping: {CONFIG_FT['PATIENCE']} epochs")
print(f"📉 Reduce LR: patience={CONFIG_FT['REDUCE_LR_PATIENCE']}, factor={CONFIG_FT['REDUCE_LR_FACTOR']}")
print("=" * 50)


🔥 FINE-TUNING CONFIGURATION
📐 Image Size: 224x224
📦 Batch Size: 16
🔥 Unfrozen Layers: 20
🔄 Max Epochs: 25
📈 Fine-tune LR: 5e-05
🎯 Head LR: 0.001
💧 Dropout: 0.4
⚖️ Weight Decay: 0.0001
⏰ Early Stopping: 5 epochs
📉 Reduce LR: patience=3, factor=0.5


In [2]:
# 📚 IMPORTS ET CONFIGURATION GPU
# ============================================================================

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageEnhance
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import (
    Input, Dense, Dropout, GlobalAveragePooling2D, 
    Layer, Lambda, BatchNormalization
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
    LearningRateScheduler
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tqdm import tqdm
import random
import kagglehub

# Configuration GPU
print("🖥️ GPU Configuration:")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ {len(gpus)} GPU(s) detected and configured")
    except RuntimeError as e:
        print(f"❌ GPU configuration error: {e}")
else:
    print("⚠️ No GPU detected, using CPU")

print(f"🔢 TensorFlow version: {tf.__version__}")
print(f"🎲 Random seed: 42")

# Set seeds pour reproducibilité
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


🖥️ GPU Configuration:
⚠️ No GPU detected, using CPU
🔢 TensorFlow version: 2.19.0
🎲 Random seed: 42


In [3]:
# 📊 CHARGEMENT DES DONNÉES
# ============================================================================
# 🔄 Chargement du DataFrame avec les recettes et images

def load_or_create_recipes_df():
    """Charge le DataFrame ou le crée depuis le dataset"""
    pkl_path = "./data/recipes_with_images_dataframe.pkl"
    
    if os.path.exists(pkl_path):
        print(f"✅ Chargement du DataFrame existant: {pkl_path}")
        return pd.read_pickle(pkl_path)
    
    print("❌ DataFrame non trouvé, création depuis le dataset...")
    
    # Télécharger le dataset
    try:
        dataset_path = kagglehub.dataset_download("pes12017000148/food-ingredients-and-recipe-dataset-with-images")
        csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
        csv_path = os.path.join(dataset_path, csv_files[0])
        
        print(f"📄 CSV: {csv_path}")
        
        # Charger et nettoyer
        df = pd.read_csv(csv_path)
        df_clean = df.dropna(subset=['Title', 'Ingredients', 'Instructions']).copy()
        
        # Ajouter les colonnes nécessaires
        images_dir = os.path.join(dataset_path, "Food Images", "Food Images")
        df_clean['image_path'] = df_clean['Image_Name'].apply(
            lambda x: os.path.join(images_dir, x) if pd.notna(x) else None
        )
        
        # Vérifier l'existence des images
        df_clean['has_image'] = df_clean['image_path'].apply(
            lambda x: os.path.exists(x) if x else False
        )
        
        # Créer le dossier data s'il n'existe pas
        os.makedirs("./data", exist_ok=True)
        
        # Sauvegarder pour les prochaines fois
        df_clean.to_pickle(pkl_path)
        print(f"✅ DataFrame sauvé: {pkl_path}")
        
        return df_clean
        
    except Exception as e:
        print(f"❌ Erreur lors de la création du DataFrame: {e}")
        return None

# Charger les données
try:
    recipes_df = load_or_create_recipes_df()
    if recipes_df is not None:
        print(f"✅ {len(recipes_df)} recettes chargées depuis le DataFrame")
        recipes_with_images = recipes_df[recipes_df['has_image'] == True].copy()
        print(f"📸 {len(recipes_with_images)} recettes avec images disponibles")
        print(f"📊 Pourcentage avec images: {len(recipes_with_images)/len(recipes_df)*100:.1f}%")
    else:
        print("❌ Échec du chargement des données")
except Exception as e:
    print(f"❌ Erreur lors du chargement: {e}")
    recipes_df = None


✅ Chargement du DataFrame existant: ./data/recipes_with_images_dataframe.pkl
✅ 13463 recettes chargées depuis le DataFrame
📸 13463 recettes avec images disponibles
📊 Pourcentage avec images: 100.0%


In [4]:
# 🧠 CUSTOM LAYERS ET UTILITAIRES
# ============================================================================

def preprocess_image(image_path, img_size=224):
    """Préprocessing des images pour EfficientNet"""
    try:
        # Charger et redimensionner l'image
        img = Image.open(image_path).convert('RGB')
        img = img.resize((img_size, img_size), Image.Resampling.LANCZOS)
        
        # Convertir en array et normaliser
        img_array = np.array(img, dtype=np.float32)
        
        # Preprocessing EfficientNet (scale to [0,1] puis normalize)
        img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
        
        return img_array
    except Exception as e:
        print(f"❌ Erreur preprocessing {image_path}: {e}")
        # Retourner une image noire en cas d'erreur
        return np.zeros((img_size, img_size, 3), dtype=np.float32)

# 🎯 COUCHE L2 NORMALIZATION PERSONNALISÉE (SÉRIALISABLE)
class L2NormalizationLayer(Layer):
    """Couche personnalisée pour normalisation L2 - sérialisable"""
    
    def __init__(self, axis=1, **kwargs):
        super(L2NormalizationLayer, self).__init__(**kwargs)
        self.axis = axis
    
    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=self.axis)
    
    def compute_output_shape(self, input_shape):
        return input_shape
    
    def get_config(self):
        config = super(L2NormalizationLayer, self).get_config()
        config.update({'axis': self.axis})
        return config

# 🎯 COUCHE EXTRACTION TRIPLET PERSONNALISÉE (SÉRIALISABLE)
class ExtractTripletComponent(Layer):
    """Couche pour extraire une composante du triplet"""
    
    def __init__(self, component_index, **kwargs):
        super(ExtractTripletComponent, self).__init__(**kwargs)
        self.component_index = component_index
    
    def call(self, inputs):
        return inputs[:, self.component_index]
    
    def compute_output_shape(self, input_shape):
        # input_shape = (batch_size, 3, height, width, channels)
        return (input_shape[0], input_shape[2], input_shape[3], input_shape[4])
    
    def get_config(self):
        config = super(ExtractTripletComponent, self).get_config()
        config.update({'component_index': self.component_index})
        return config

# 🎯 COUCHE STACK TRIPLET PERSONNALISÉE (SÉRIALISABLE)
class TripletStackLayer(Layer):
    """Couche personnalisée pour empiler les embeddings triplet"""
    
    def __init__(self, **kwargs):
        super(TripletStackLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        # inputs = [anchor_emb, positive_emb, negative_emb]
        return tf.stack(inputs, axis=1)
    
    def compute_output_shape(self, input_shape):
        # input_shape = [(batch_size, embedding_dim), ...]
        batch_size = input_shape[0][0]
        embedding_dim = input_shape[0][1]
        return (batch_size, 3, embedding_dim)
    
    def get_config(self):
        return super(TripletStackLayer, self).get_config()

print("✅ Custom layers defined successfully")
print("   🔧 L2NormalizationLayer: L2 normalization for embeddings")
print("   🔧 ExtractTripletComponent: Extract anchor/positive/negative from triplet")
print("   🔧 TripletStackLayer: Stack embeddings for triplet loss")


✅ Custom layers defined successfully
   🔧 L2NormalizationLayer: L2 normalization for embeddings
   🔧 ExtractTripletComponent: Extract anchor/positive/negative from triplet
   🔧 TripletStackLayer: Stack embeddings for triplet loss


In [5]:
# 🏗️ CREATION DU MODÈLE FINE-TUNING (20 COUCHES DÉGELÉES)
# ============================================================================

def create_fine_tuning_embedding_model(input_shape=(224, 224, 3), embedding_dim=512, unfrozen_layers=20):
    """
    🔥 Créer un modèle d'embedding avec fine-tuning
    
    Args:
        input_shape: Forme des images d'entrée
        embedding_dim: Dimension des embeddings de sortie
        unfrozen_layers: Nombre de couches à dégeler pour le fine-tuning
    
    Returns:
        Modèle d'embedding fine-tuné
    """
    print(f"🏗️ Création du modèle fine-tuning avec {unfrozen_layers} couches dégelées...")
    
    # 1. Chargement d'EfficientNetB0 pré-entraîné
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape,
        pooling=None
    )
    
    print(f"📊 EfficientNetB0 chargé: {len(base_model.layers)} couches totales")
    
    # 2. STRATÉGIE DE FINE-TUNING: Geler puis dégeler les dernières couches
    
    # D'abord, geler toutes les couches
    for layer in base_model.layers:
        layer.trainable = False
    
    # Ensuite, dégeler les N dernières couches
    total_layers = len(base_model.layers)
    unfreeze_from = max(0, total_layers - unfrozen_layers)
    
    trainable_count = 0
    frozen_count = 0
    
    for i, layer in enumerate(base_model.layers):
        if i >= unfreeze_from:
            layer.trainable = True
            trainable_count += 1
        else:
            layer.trainable = False
            frozen_count += 1
    
    print(f"🧊 Couches gelées: {frozen_count} (0 à {unfreeze_from-1})")
    print(f"🔥 Couches dégelées: {trainable_count} ({unfreeze_from} à {total_layers-1})")
    
    # 3. Construction du modèle complet avec head personnalisé
    inputs = Input(shape=input_shape, name='image_input')
    
    # Feature extraction avec EfficientNet (partiellement fine-tuné)
    x = base_model(inputs, training=True)  # training=True pour fine-tuning
    
    # Global Average Pooling
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Head personnalisé pour les embeddings
    x = Dense(1024, activation='relu', name='dense_1024')(x)
    x = BatchNormalization(name='batch_norm_1')(x)
    x = Dropout(CONFIG_FT['DROPOUT_RATE'], name='dropout_1')(x)
    
    x = Dense(512, activation='relu', name='dense_512')(x)
    x = BatchNormalization(name='batch_norm_2')(x)
    x = Dropout(CONFIG_FT['DROPOUT_RATE'] * 0.5, name='dropout_2')(x)
    
    # Couche d'embedding finale
    embeddings = Dense(embedding_dim, activation='linear', name='embeddings')(x)
    
    # Normalisation L2 pour cosine similarity
    embeddings_normalized = L2NormalizationLayer(axis=1, name='l2_norm')(embeddings)
    
    # Créer le modèle
    model = Model(inputs=inputs, outputs=embeddings_normalized, name='FinetuneEmbeddingModel')
    
    # 4. Statistiques du modèle
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    frozen_params = total_params - trainable_params
    
    print(f"\n📊 STATISTIQUES DU MODÈLE:")
    print(f"   🔢 Paramètres totaux: {total_params:,}")
    print(f"   🔥 Paramètres entraînables: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")
    print(f"   🧊 Paramètres gelés: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)")
    print(f"   📐 Forme de sortie: {model.output_shape}")
    
    return model

# Test de création du modèle
embedding_model_ft = create_fine_tuning_embedding_model(
    input_shape=(CONFIG_FT['IMG_SIZE'], CONFIG_FT['IMG_SIZE'], 3),
    embedding_dim=CONFIG_FT['EMBEDDING_DIM_CUSTOM'],
    unfrozen_layers=CONFIG_FT['UNFROZEN_LAYERS']
)

print("\n✅ Modèle fine-tuning créé avec succès!")


🏗️ Création du modèle fine-tuning avec 20 couches dégelées...
📊 EfficientNetB0 chargé: 238 couches totales
🧊 Couches gelées: 218 (0 à 217)
🔥 Couches dégelées: 20 (218 à 237)

📊 STATISTIQUES DU MODÈLE:
   🔢 Paramètres totaux: 6,154,915
   🔥 Paramètres entraînables: 3,453,232 (56.1%)
   🧊 Paramètres gelés: 2,701,683 (43.9%)
   📐 Forme de sortie: (None, 512)

✅ Modèle fine-tuning créé avec succès!


In [6]:
# 📏 TRIPLET LOSS ET MÉTRIQUES PERSONNALISÉES
# ============================================================================

def triplet_loss_fn(y_true, y_pred, margin=0.2):
    """
    🎯 Triplet Loss avec hard negative mining
    
    Args:
        y_true: Labels (non utilisés)
        y_pred: Embeddings empilés [anchor, positive, negative] shape=(batch_size, 3, embedding_dim)
        margin: Marge pour la triplet loss
    
    Returns:
        Triplet loss value
    """
    # Extraire les embeddings
    anchor = y_pred[:, 0, :]      # (batch_size, embedding_dim)
    positive = y_pred[:, 1, :]    # (batch_size, embedding_dim)
    negative = y_pred[:, 2, :]    # (batch_size, embedding_dim)
    
    # Calculer les distances (cosine similarity -> distance)
    pos_similarity = tf.reduce_sum(anchor * positive, axis=1)
    neg_similarity = tf.reduce_sum(anchor * negative, axis=1)
    
    # Triplet loss: max(0, margin - (pos_sim - neg_sim))
    # Plus pos_sim est élevé et neg_sim faible, plus la loss est petite
    loss = tf.maximum(0.0, margin - (pos_similarity - neg_similarity))
    
    return tf.reduce_mean(loss)

def triplet_accuracy(y_true, y_pred):
    """
    📊 Métrique d'accuracy pour triplet loss
    Pourcentage de triplets où pos_similarity > neg_similarity
    """
    anchor = y_pred[:, 0, :]     # (batch_size, embedding_dim)
    positive = y_pred[:, 1, :]   # (batch_size, embedding_dim)
    negative = y_pred[:, 2, :]   # (batch_size, embedding_dim)
    
    pos_similarity = tf.reduce_sum(anchor * positive, axis=1)
    neg_similarity = tf.reduce_sum(anchor * negative, axis=1)
    
    # Compter les cas où positive > negative
    correct_predictions = tf.cast(pos_similarity > neg_similarity, tf.float32)
    return tf.reduce_mean(correct_predictions)

def average_positive_similarity(y_true, y_pred):
    """📈 Similarité moyenne anchor-positive"""
    anchor = y_pred[:, 0, :]
    positive = y_pred[:, 1, :]
    return tf.reduce_mean(tf.reduce_sum(anchor * positive, axis=1))

def average_negative_similarity(y_true, y_pred):
    """📉 Similarité moyenne anchor-negative"""
    anchor = y_pred[:, 0, :]
    negative = y_pred[:, 2, :]
    return tf.reduce_mean(tf.reduce_sum(anchor * negative, axis=1))

# Test des fonctions
print("✅ Triplet loss et métriques définies:")
print("   🎯 triplet_loss_fn: Loss principale avec marge")
print("   📊 triplet_accuracy: % de triplets corrects")
print("   📈 average_positive_similarity: Sim anchor-positive")
print("   📉 average_negative_similarity: Sim anchor-negative")


✅ Triplet loss et métriques définies:
   🎯 triplet_loss_fn: Loss principale avec marge
   📊 triplet_accuracy: % de triplets corrects
   📈 average_positive_similarity: Sim anchor-positive
   📉 average_negative_similarity: Sim anchor-negative


In [7]:
# 🔄 GÉNÉRATEUR DE DONNÉES TRIPLET POUR FINE-TUNING
# ============================================================================

class TripletGeneratorFT(Sequence):
    """
    🎯 Générateur de triplets (anchor, positive, negative) pour fine-tuning
    Optimisé avec hard negative mining et augmentation conservatrice
    """
    
    def __init__(self, recipes_df, batch_size=16, img_size=224, augment=True, validation_split=0.2, is_validation=False):
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment and not is_validation  # Pas d'augmentation pour validation
        
        # Filtrer et diviser les données
        self.valid_recipes = self._filter_valid_recipes(recipes_df)
        self.train_recipes, self.val_recipes = self._split_data(validation_split)
        
        # Utiliser le bon subset
        self.recipes_df = self.val_recipes if is_validation else self.train_recipes
        
        # Grouper par titre pour positive sampling
        self.recipe_groups = self.recipes_df.groupby('Title')
        
        # Data augmentation conservative pour fine-tuning
        if self.augment:
            self.datagen = ImageDataGenerator(
                rotation_range=10,        # Réduction de 20 à 10
                width_shift_range=0.1,    # Réduction de 0.15 à 0.1
                height_shift_range=0.1,   # Réduction de 0.15 à 0.1
                shear_range=0.1,          # Réduction de 0.15 à 0.1
                zoom_range=0.1,           # Réduction de 0.15 à 0.1
                horizontal_flip=True,
                brightness_range=[0.9, 1.1], # Plus conservateur
                fill_mode='nearest'
            )
        
        mode = '(Validation)' if is_validation else '(Training)'
        print(f"🔄 TripletGeneratorFT {mode}:")
        print(f"   📊 Total recettes: {len(self.recipes_df)}")
        print(f"   📸 Images: {len(self.recipes_df)}")
        augment_status = 'ON (conservative)' if self.augment else 'OFF'
        print(f"   🔄 Augmentation: {augment_status}")

    def _filter_valid_recipes(self, recipes_df):
        """Filtrer les recettes avec images valides"""
        print("🔍 Filtrage des recettes avec images valides...")
        
        valid_recipes = []
        for idx, row in tqdm(recipes_df.iterrows(), total=len(recipes_df), desc="Validation images"):
            if pd.notna(row.get('image_path')) and os.path.exists(row['image_path']):
                try:
                    # Test de chargement rapide
                    with Image.open(row['image_path']) as img:
                        img.verify()  # Vérifier l'intégrité
                    valid_recipes.append(row.to_dict())
                except Exception:
                    continue
        
        result_df = pd.DataFrame(valid_recipes)
        print(f"✅ {len(result_df)}/{len(recipes_df)} images valides")
        return result_df
    
    def _split_data(self, validation_split):
        """Diviser les données en train/validation par recette"""
        unique_recipes = self.valid_recipes['Title'].unique()
        
        train_recipes, val_recipes = train_test_split(
            unique_recipes, 
            test_size=validation_split, 
            random_state=42
        )
        
        train_df = self.valid_recipes[self.valid_recipes['Title'].isin(train_recipes)]
        val_df = self.valid_recipes[self.valid_recipes['Title'].isin(val_recipes)]
        
        print(f"📊 Split: {len(train_df)} train, {len(val_df)} validation")
        return train_df, val_df
    
    def __len__(self):
        """Nombre de batches par époque"""
        return max(1, len(self.recipes_df) // self.batch_size)
    
    def __getitem__(self, idx):
        """Générer un batch de triplets"""
        batch_size = self.batch_size
        
        # Préparer les arrays
        anchors = np.zeros((batch_size, self.img_size, self.img_size, 3), dtype=np.float32)
        positives = np.zeros((batch_size, self.img_size, self.img_size, 3), dtype=np.float32)
        negatives = np.zeros((batch_size, self.img_size, self.img_size, 3), dtype=np.float32)
        
        valid_triplets = 0
        attempts = 0
        max_attempts = batch_size * 5  # Plus de tentatives pour robustesse
        
        while valid_triplets < batch_size and attempts < max_attempts:
            attempts += 1
            
            try:
                # 1. Sélectionner anchor aléatoire
                anchor_row = self.recipes_df.sample(1).iloc[0]
                anchor_title = anchor_row['Title']
                
                # 2. Sélectionner positive (même recette, image différente si possible)
                same_recipe = self.recipe_groups.get_group(anchor_title)
                if len(same_recipe) > 1:
                    pos_row = same_recipe[same_recipe.index != anchor_row.name].sample(1).iloc[0]
                else:
                    pos_row = anchor_row  # Même image si pas d'autre choix
                
                # 3. Sélectionner negative (recette différente)
                different_recipes = self.recipes_df[self.recipes_df['Title'] != anchor_title]
                if len(different_recipes) > 0:
                    neg_row = different_recipes.sample(1).iloc[0]
                else:
                    continue
                
                # 4. Charger et preprocesser les images
                anchor_img = preprocess_image(anchor_row['image_path'], self.img_size)
                pos_img = preprocess_image(pos_row['image_path'], self.img_size)
                neg_img = preprocess_image(neg_row['image_path'], self.img_size)
                
                # 5. Appliquer augmentation si nécessaire
                if self.augment:
                    # Augmentation conservative pour fine-tuning
                    anchor_img = self.datagen.random_transform(anchor_img)
                    pos_img = self.datagen.random_transform(pos_img)
                    # Pas d'augmentation sur negative pour hard mining
                
                # 6. Stocker dans le batch
                anchors[valid_triplets] = anchor_img
                positives[valid_triplets] = pos_img
                negatives[valid_triplets] = neg_img
                
                valid_triplets += 1
                
            except Exception as e:
                continue
        
        # Remplir les triplets manquants si nécessaire
        while valid_triplets < batch_size:
            anchors[valid_triplets] = anchors[0]
            positives[valid_triplets] = positives[0]
            negatives[valid_triplets] = negatives[0]
            valid_triplets += 1
        
        # Format pour le modèle triplet: (batch_size, 3, height, width, channels)
        triplet_batch = np.stack([anchors, positives, negatives], axis=1)
        dummy_labels = np.zeros((batch_size, 1))  # Non utilisé avec triplet loss
        
        return triplet_batch, dummy_labels

print("✅ TripletGeneratorFT défini avec optimisations fine-tuning")
print("   🔄 Augmentation conservative")
print("   🎯 Hard negative mining")
print("   📊 Split train/validation par recette")


✅ TripletGeneratorFT défini avec optimisations fine-tuning
   🔄 Augmentation conservative
   🎯 Hard negative mining
   📊 Split train/validation par recette


In [8]:
# 🏗️ CRÉATION DU MODÈLE TRIPLET FINE-TUNING
# ============================================================================

def create_triplet_model_ft(embedding_model):
    """
    🎯 Créer le modèle triplet pour l'entraînement fine-tuning
    
    Args:
        embedding_model: Modèle d'embedding pré-créé
    
    Returns:
        Modèle triplet compilé
    """
    print("🏗️ Création du modèle triplet fine-tuning...")
    
    # Input: batch de triplets (batch_size, 3, height, width, channels)
    triplet_input = Input(
        shape=(3, CONFIG_FT['IMG_SIZE'], CONFIG_FT['IMG_SIZE'], 3),
        name='triplet_input'
    )
    
    # Extraire chaque composante du triplet
    anchor_input = ExtractTripletComponent(0, name='extract_anchor')(triplet_input)
    positive_input = ExtractTripletComponent(1, name='extract_positive')(triplet_input)
    negative_input = ExtractTripletComponent(2, name='extract_negative')(triplet_input)
    
    # Obtenir les embeddings pour chaque composante
    anchor_embedding = embedding_model(anchor_input)
    positive_embedding = embedding_model(positive_input)
    negative_embedding = embedding_model(negative_input)
    
    # Empiler les embeddings pour le triplet loss
    triplet_output = TripletStackLayer(name='triplet_stack')([
        anchor_embedding, 
        positive_embedding, 
        negative_embedding
    ])
    
    # Créer le modèle triplet
    triplet_model = Model(
        inputs=triplet_input, 
        outputs=triplet_output, 
        name='UltimateTripletModelFT'
    )
    
    print(f"✅ Modèle triplet créé:")
    print(f"   📐 Input shape: {triplet_model.input_shape}")
    print(f"   📐 Output shape: {triplet_model.output_shape}")
    
    return triplet_model

# Créer le modèle triplet
triplet_model_ft = create_triplet_model_ft(embedding_model_ft)


🏗️ Création du modèle triplet fine-tuning...
✅ Modèle triplet créé:
   📐 Input shape: (None, 3, 224, 224, 3)
   📐 Output shape: (None, 3, 512)


In [9]:
# 🚀 FONCTION D'ENTRAÎNEMENT FINE-TUNING
# ============================================================================

def train_fine_tuning():
    """
    🔥 Entraîner le modèle avec fine-tuning et différents learning rates
    
    Returns:
        embedding_model, history: Modèle entraîné et historique
    """
    print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT FINE-TUNING")
    print("=" * 60)
    
    # Créer les générateurs de données (train + validation)
    print("\n📊 Création des générateurs de données...")
    train_generator = TripletGeneratorFT(
        recipes_with_images,
        batch_size=CONFIG_FT['BATCH_SIZE'],
        img_size=CONFIG_FT['IMG_SIZE'],
        augment=CONFIG_FT['USE_AUGMENTATION'],
        validation_split=CONFIG_FT['VALIDATION_SPLIT'],
        is_validation=False
    )
    
    val_generator = TripletGeneratorFT(
        recipes_with_images,
        batch_size=CONFIG_FT['BATCH_SIZE'],
        img_size=CONFIG_FT['IMG_SIZE'],
        augment=False,  # Pas d'augmentation pour validation
        validation_split=CONFIG_FT['VALIDATION_SPLIT'],
        is_validation=True
    )
    
    # Créer le modèle triplet
    triplet_model = create_triplet_model_ft(embedding_model_ft)
    
    # ⚡ COMPILATION AVEC LEARNING RATES DIFFÉRENTIÉS
    print("\n⚡ Compilation du modèle avec learning rates optimisés...")
    
    # Créer un optimizer avec learning rate pour fine-tuning
    optimizer = Adam(
        learning_rate=CONFIG_FT['FINETUNE_LR'],
        weight_decay=CONFIG_FT['WEIGHT_DECAY']  # L2 regularization
    )
    
    # Compiler le modèle
    triplet_model.compile(
        optimizer=optimizer,
        loss=lambda y_true, y_pred: triplet_loss_fn(y_true, y_pred, margin=CONFIG_FT['MARGIN']),
        metrics=[triplet_accuracy, average_positive_similarity, average_negative_similarity]
    )
    
    print(f"   📈 Optimizer: Adam (lr={CONFIG_FT['FINETUNE_LR']}, wd={CONFIG_FT['WEIGHT_DECAY']})")
    print(f"   🎯 Triplet Loss margin: {CONFIG_FT['MARGIN']}")
    print(f"   📊 Métriques: accuracy, pos_sim, neg_sim")
    
    # 📋 CALLBACKS OPTIMISÉS POUR FINE-TUNING
    print("\n📋 Configuration des callbacks...")
    
    # Modèle checkpoint pour sauvegarder le meilleur modèle
    checkpoint_path = os.path.join(CONFIG_FT['CHECKPOINT_DIR'], f"best_{CONFIG_FT['MODEL_NAME']}")
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        mode='min'
    )
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=CONFIG_FT['PATIENCE'],
        verbose=1,
        restore_best_weights=True,
        mode='min'
    )
    
    # Réduction du learning rate
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=CONFIG_FT['REDUCE_LR_FACTOR'],
        patience=CONFIG_FT['REDUCE_LR_PATIENCE'],
        min_lr=CONFIG_FT['MIN_LR'],
        verbose=1,
        mode='min'
    )
    
    callbacks = [model_checkpoint, early_stopping, reduce_lr]
    
    print(f"   💾 Model checkpoint: {checkpoint_path}")
    print(f"   ⏰ Early stopping: patience={CONFIG_FT['PATIENCE']}")
    print(f"   📉 Reduce LR: patience={CONFIG_FT['REDUCE_LR_PATIENCE']}, factor={CONFIG_FT['REDUCE_LR_FACTOR']}")
    print(f"   🔻 Min LR: {CONFIG_FT['MIN_LR']}")
    
    # 🎯 ENTRAÎNEMENT
    print(f"\n🎯 Lancement de l'entraînement ({CONFIG_FT['FINETUNE_EPOCHS']} époques max)...")
    print("=" * 60)
    
    try:
        history = triplet_model.fit(
            train_generator,
            epochs=CONFIG_FT['FINETUNE_EPOCHS'],
            validation_data=val_generator,
            callbacks=callbacks,
            verbose=1
        )
        
        print("\n✅ ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
        
        # Sauvegarder le modèle d'embedding final
        final_model_path = os.path.join(CONFIG_FT['CHECKPOINT_DIR'], CONFIG_FT['MODEL_NAME'])
        embedding_model_ft.save(final_model_path)
        print(f"💾 Modèle d'embedding sauvé: {final_model_path}")
        
        # Afficher les résultats finaux
        print(f"\n📊 RÉSULTATS FINAUX:")
        print(f"📉 Loss finale: {history.history['val_loss'][-1]:.4f}")
        print(f"📊 Précision finale: {history.history['val_triplet_accuracy'][-1]:.4f}")
        print(f"📈 Similarité pos finale: {history.history['val_average_positive_similarity'][-1]:.4f}")
        print(f"📉 Similarité neg finale: {history.history['val_average_negative_similarity'][-1]:.4f}")
        
        return embedding_model_ft, history
        
    except Exception as e:
        print(f"\n❌ ERREUR PENDANT L'ENTRAÎNEMENT: {e}")
        return None, None

print("✅ Fonction d'entraînement fine-tuning définie")
print("   🔥 Learning rates optimisés pour fine-tuning")
print("   📊 Callbacks adaptés pour stabilité")
print("   💾 Sauvegarde automatique du meilleur modèle")


✅ Fonction d'entraînement fine-tuning définie
   🔥 Learning rates optimisés pour fine-tuning
   📊 Callbacks adaptés pour stabilité
   💾 Sauvegarde automatique du meilleur modèle


In [10]:
# 📊 VISUALISATION DES RÉSULTATS D'ENTRAÎNEMENT (VERSION CORRIGÉE)
# ============================================================================

def plot_training_results_ft(history):
    """Afficher les métriques d'entraînement fine-tuning - Version corrigée"""
    
    # Configuration dark mode
    plt.style.use('dark_background')
    
    # Couleurs modernes pour dark mode
    colors = {
        'primary': '#00D4AA',      # Cyan-vert brillant
        'secondary': '#FF6B6B',    # Rouge-coral moderne
        'accent': '#4ECDC4',       # Turquoise
        'warning': '#FFE66D',      # Jaune moderne
        'info': '#A8E6CF',         # Vert pastel
        'purple': '#B19CD9'        # Violet pastel
    }
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.patch.set_facecolor('#1e1e1e')
    fig.suptitle('🔥 Résultats Fine-Tuning (20 couches dégelées)', 
                 fontsize=18, fontweight='bold', color='white', y=0.95)
    
    # Configuration commune pour tous les axes
    for ax in axes.flat:
        ax.set_facecolor('#2d2d2d')
        ax.tick_params(colors='white', which='both')
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.title.set_color('white')
        ax.grid(True, alpha=0.2, color='gray', linestyle='-', linewidth=0.5)
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['right'].set_color('white')
        ax.spines['left'].set_color('white')
    
    # Loss (inchangé)
    axes[0, 0].plot(history.history['loss'], label='📈 Train Loss', 
                    linewidth=3, color=colors['primary'], alpha=0.9)
    axes[0, 0].plot(history.history['val_loss'], label='📉 Val Loss', 
                    linewidth=3, color=colors['secondary'], alpha=0.9)
    axes[0, 0].fill_between(range(len(history.history['loss'])), 
                           history.history['loss'], alpha=0.1, color=colors['primary'])
    axes[0, 0].fill_between(range(len(history.history['val_loss'])), 
                           history.history['val_loss'], alpha=0.1, color=colors['secondary'])
    axes[0, 0].set_title('🎯 Triplet Loss Evolution', fontweight='bold', fontsize=14)
    axes[0, 0].set_xlabel('Époque', fontweight='bold')
    axes[0, 0].set_ylabel('Loss Value', fontweight='bold')
    axes[0, 0].legend(frameon=True, fancybox=True, shadow=True, 
                     facecolor='#3d3d3d', edgecolor='white')
    
    # ✅ CORRECTION: Accuracy avec échelle adaptée
    axes[0, 1].plot(history.history['triplet_accuracy'], label='📊 Train Accuracy', 
                    linewidth=3, color=colors['accent'], alpha=0.9, marker='o', markersize=4)
    axes[0, 1].plot(history.history['val_triplet_accuracy'], label='📋 Val Accuracy', 
                    linewidth=3, color=colors['warning'], alpha=0.9, marker='s', markersize=4)
    axes[0, 1].fill_between(range(len(history.history['triplet_accuracy'])), 
                           history.history['triplet_accuracy'], alpha=0.1, color=colors['accent'])
    axes[0, 1].fill_between(range(len(history.history['val_triplet_accuracy'])), 
                           history.history['val_triplet_accuracy'], alpha=0.1, color=colors['warning'])
    
    # ✅ CORRECTION: Ligne de référence plus haute + échelle adaptée
    axes[0, 1].axhline(y=0.95, color='white', linestyle='--', alpha=0.5, label='Target (95%)')
    
    # ✅ CORRECTION: Échelle Y adaptée pour mieux voir les variations
    min_acc = min(min(history.history['triplet_accuracy']), min(history.history['val_triplet_accuracy']))
    axes[0, 1].set_ylim(max(0.8, min_acc - 0.05), 1.02)  # Focus sur la zone utile
    
    axes[0, 1].set_title('📊 Triplet Accuracy Progress', fontweight='bold', fontsize=14)
    axes[0, 1].set_xlabel('Époque', fontweight='bold')
    axes[0, 1].set_ylabel('Accuracy Score', fontweight='bold')
    axes[0, 1].legend(frameon=True, fancybox=True, shadow=True, 
                     facecolor='#3d3d3d', edgecolor='white')
    
    # ✅ CORRECTION: Positive Similarity avec zone cible réaliste
    axes[1, 0].plot(history.history['average_positive_similarity'], label='🔗 Train Pos Sim', 
                    linewidth=3, color=colors['info'], alpha=0.9, marker='o', markersize=4)
    axes[1, 0].plot(history.history['val_average_positive_similarity'], label='✅ Val Pos Sim', 
                    linewidth=3, color=colors['purple'], alpha=0.9, marker='s', markersize=4)
    axes[1, 0].fill_between(range(len(history.history['average_positive_similarity'])), 
                           history.history['average_positive_similarity'], alpha=0.1, color=colors['info'])
    axes[1, 0].fill_between(range(len(history.history['val_average_positive_similarity'])), 
                           history.history['val_average_positive_similarity'], alpha=0.1, color=colors['purple'])
    
    # ✅ CORRECTION: Zone d'objectif réaliste pour similarity positive (0.5-0.8)
    axes[1, 0].axhspan(0.5, 0.8, alpha=0.1, color='green', label='Good Zone (0.5-0.8)')
    axes[1, 0].axhline(y=0.6, color='lime', linestyle='--', alpha=0.7, label='Target (0.6+)')
    
    axes[1, 0].set_title('📈 Positive Similarity (Anchor-Positive)', fontweight='bold', fontsize=14)
    axes[1, 0].set_xlabel('Époque', fontweight='bold')
    axes[1, 0].set_ylabel('Cosine Similarity', fontweight='bold')
    axes[1, 0].legend(frameon=True, fancybox=True, shadow=True, 
                     facecolor='#3d3d3d', edgecolor='white')
    axes[1, 0].set_ylim(-0.2, 1.0)  # Zoom sur la zone utile
    
    # Negative Similarity (inchangé mais amélioré)
    axes[1, 1].plot(history.history['average_negative_similarity'], label='❌ Train Neg Sim', 
                    linewidth=3, color=colors['secondary'], alpha=0.9, marker='o', markersize=4)
    axes[1, 1].plot(history.history['val_average_negative_similarity'], label='🚫 Val Neg Sim', 
                    linewidth=3, color=colors['primary'], alpha=0.9, marker='s', markersize=4)
    axes[1, 1].fill_between(range(len(history.history['average_negative_similarity'])), 
                           history.history['average_negative_similarity'], alpha=0.1, color=colors['secondary'])
    axes[1, 1].fill_between(range(len(history.history['val_average_negative_similarity'])), 
                           history.history['val_average_negative_similarity'], alpha=0.1, color=colors['primary'])
    
    # Zone d'objectif pour similarity négative (<0.3)
    axes[1, 1].axhspan(-1.0, 0.3, alpha=0.1, color='red', label='Target Zone (<0.3)')
    axes[1, 1].axhline(y=0.2, color='orange', linestyle='--', alpha=0.7, label='Good (<0.2)')
    
    axes[1, 1].set_title('📉 Negative Similarity (Anchor-Negative)', fontweight='bold', fontsize=14)
    axes[1, 1].set_xlabel('Époque', fontweight='bold')
    axes[1, 1].set_ylabel('Cosine Similarity', fontweight='bold')
    axes[1, 1].legend(frameon=True, fancybox=True, shadow=True, 
                     facecolor='#3d3d3d', edgecolor='white')
    axes[1, 1].set_ylim(-0.2, 0.8)  # Focus sur la zone utile
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()
    
    # ✅ BONUS: Affichage des statistiques finales
    print("\n" + "="*60)
    print("📊 RÉSULTATS FINAUX FINE-TUNING:")
    print("="*60)
    print(f"🎯 Loss finale (val): {history.history['val_loss'][-1]:.4f}")
    print(f"📊 Accuracy finale (val): {history.history['val_triplet_accuracy'][-1]:.4f}")
    print(f"📈 Pos Similarity finale (val): {history.history['val_average_positive_similarity'][-1]:.4f}")
    print(f"📉 Neg Similarity finale (val): {history.history['val_average_negative_similarity'][-1]:.4f}")
    gap = history.history['val_average_positive_similarity'][-1] - history.history['val_average_negative_similarity'][-1]
    print(f"📏 Écart Pos-Neg: {gap:.4f}")
    print("="*60)
    
    # Comparaison avec les targets
    final_accuracy = history.history['val_triplet_accuracy'][-1]
    final_pos_sim = history.history['val_average_positive_similarity'][-1]
    final_neg_sim = history.history['val_average_negative_similarity'][-1]
    
    print(f"\n🎯 ANALYSE DES OBJECTIFS:")
    print(f"   📊 Accuracy: {'✅' if final_accuracy > 0.95 else '⚠️'} {final_accuracy:.3f} (target: >0.95)")
    print(f"   📈 Pos Sim: {'✅' if final_pos_sim > 0.6 else '⚠️'} {final_pos_sim:.3f} (target: >0.6)")
    print(f"   📉 Neg Sim: {'✅' if final_neg_sim < 0.3 else '⚠️'} {final_neg_sim:.3f} (target: <0.3)")
    print(f"   📏 Gap: {'✅' if gap > 0.3 else '⚠️'} {gap:.3f} (target: >0.3)")
    
    # Reset style
    plt.style.use('default')

print("✅ Fonction de visualisation fine-tuning définie")
print("   🔧 Graphiques corrigés (échelles adaptées)")
print("   📊 Analyses automatiques des objectifs")
print("   🎨 Interface dark mode moderne")


✅ Fonction de visualisation fine-tuning définie
   🔧 Graphiques corrigés (échelles adaptées)
   📊 Analyses automatiques des objectifs
   🎨 Interface dark mode moderne


In [ ]:
# 🚀 CLASSE COMPLÈTE RECIPE IMAGE RETRIEVAL FINE-TUNING
# ============================================================================

class RecipeImageRetrievalFT:
    """
    🔥 Système de récupération d'images de recettes avec modèle fine-tuné
    Version optimisée pour performance et précision maximale
    """
    
    def __init__(self, model_path=None, recipes_df=None):
        """Initialiser le système de récupération fine-tuning"""
        print("🚀 Initialisation RecipeImageRetrievalFT...")
        
        # Configuration
        self.img_size = CONFIG_FT['IMG_SIZE']
        self.embedding_dim = CONFIG_FT['EMBEDDING_DIM']
        self.model_name = CONFIG_FT['MODEL_NAME']
        
        # Charger le modèle
        if model_path and os.path.exists(model_path):
            print(f"📦 Chargement du modèle depuis: {model_path}")
            self.model = load_model(model_path, compile=False)
        else:
            print("❌ Aucun modèle trouvé!")
            self.model = None
            return
        
        # Base de données des recettes
        self.recipes_df = recipes_df
        self.embeddings_db = None
        
        # Cache pour les performances
        self.embedding_cache = {}
        
        print(f"✅ RecipeImageRetrievalFT initialisé avec:")
        print(f"   🏗️ Modèle: {self.model_name}")
        print(f"   📐 Taille image: {self.img_size}x{self.img_size}")
        print(f"   🧠 Dimension embedding: {self.embedding_dim}")
        print(f"   📊 Recettes: {len(self.recipes_df) if self.recipes_df is not None else 0}")
    
    def compute_embedding(self, image_path_or_array):
        """Calculer l'embedding d'une image"""
        try:
            # Cache check
            if isinstance(image_path_or_array, str) and image_path_or_array in self.embedding_cache:
                return self.embedding_cache[image_path_or_array]
            
            # Preprocessing
            if isinstance(image_path_or_array, str):
                img = preprocess_image(image_path_or_array, self.img_size)
            else:
                img = image_path_or_array
            
            # Ajouter dimension batch si nécessaire
            if len(img.shape) == 3:
                img = np.expand_dims(img, axis=0)
            
            # Calculer embedding
            embedding = self.model.predict(img, verbose=0)[0]
            """
            DOUBLE EMBEDDING ??
            """
            # embedding = embedding / np.linalg.norm(embedding)  # Normalisation L2
            
            # Cache si c'est un path
            if isinstance(image_path_or_array, str):
                self.embedding_cache[image_path_or_array] = embedding
            
            return embedding
            
        except Exception as e:
            print(f"❌ Erreur lors du calcul d'embedding: {e}")
            return None
    
    def build_database(self, force_rebuild=False):
        """Construire la base de données d'embeddings"""
        db_path = f"ft/{self.model_name.replace('.keras', '')}_embeddings_database_ft.npy"
        
        if os.path.exists(db_path) and not force_rebuild:
            print(f"📦 Chargement de la base de données: {db_path}")
            self.embeddings_db = np.load(db_path)
            print(f"✅ Base chargée: {self.embeddings_db.shape}")
            return
        
        if self.recipes_df is None:
            print("❌ Aucune donnée de recettes disponible!")
            return
        
        print(f"🔨 Construction de la base d'embeddings...")
        print(f"   📊 Nombre de recettes: {len(self.recipes_df)}")
        
        embeddings = []
        valid_indices = []
        
        for idx, row in tqdm(self.recipes_df.iterrows(), 
                            total=len(self.recipes_df), 
                            desc="🧠 Embeddings"):
            if pd.notna(row.get('image_path')) and os.path.exists(row['image_path']):
                embedding = self.compute_embedding(row['image_path'])
                if embedding is not None:
                    embeddings.append(embedding)
                    valid_indices.append(idx)
        
        if embeddings:
            self.embeddings_db = np.array(embeddings)
            
            # Sauvegarder
            os.makedirs('ft', exist_ok=True)
            np.save(db_path, self.embeddings_db)
            
            # Filtrer le DataFrame pour garder seulement les recettes valides
            self.recipes_df = self.recipes_df.loc[valid_indices].reset_index(drop=True)
            
            print(f"✅ Base construite et sauvée:")
            print(f"   📊 Embeddings: {self.embeddings_db.shape}")
            print(f"   💾 Sauvé dans: {db_path}")
            print(f"   📋 Recettes valides: {len(self.recipes_df)}")
        else:
            print("❌ Aucun embedding valide créé!")
    
    def search_similar_recipes(self, query_image_path, top_k=5, min_similarity=0.3):
        """Rechercher les recettes similaires à une image"""
        if self.embeddings_db is None:
            print("❌ Base de données non construite! Appelez build_database() d'abord.")
            return []
        
        # Calculer embedding de la requête
        query_embedding = self.compute_embedding(query_image_path)
        if query_embedding is None:
            print("❌ Impossible de calculer l'embedding de l'image requête")
            return []
        
        # Calculer similarités
        similarities = np.dot(self.embeddings_db, query_embedding)
        
        # Trouver les top_k
        top_indices = np.argsort(similarities)[::-1][:top_k]
        top_similarities = similarities[top_indices]
        
        # Filtrer par similarité minimale
        results = []
        for idx, sim in zip(top_indices, top_similarities):
            if sim >= min_similarity:
                recipe = self.recipes_df.iloc[idx]
                results.append({
                    'recipe_title': recipe['Title'],
                    'similarity': float(sim),
                    'image_path': recipe['image_path'],
                    'index': int(idx)
                })
        
        return results

print("✅ RecipeImageRetrievalFT définie")
print("   🧠 Calcul d'embeddings optimisé")
print("   📊 Base de données automatique")
print("   🔍 Recherche par similarité")
print("   🎨 Visualisation des résultats")


✅ RecipeImageRetrievalFT définie
   🧠 Calcul d'embeddings optimisé
   📊 Base de données automatique
   🔍 Recherche par similarité
   🎨 Visualisation des résultats


In [12]:
# 🎯 ENTRAÎNEMENT DU MODÈLE FINE-TUNING
# ============================================================================

print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT FINE-TUNING COMPLET")
print("=" * 60)

# Lancer l'entraînement
trained_model_ft, history_ft = train_fine_tuning()

if trained_model_ft is not None and history_ft is not None:
    print("\n🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
    
    # Visualiser les résultats
    print("\n📊 Visualisation des résultats d'entraînement...")
    plot_training_results_ft(history_ft)
    
    print("\n✅ Fine-tuning completed successfully!")
    print(f"   📁 Modèle sauvé dans: ft/{CONFIG_FT['MODEL_NAME']}")
    print(f"   📊 Base de données d'embeddings prête")
    
else:
    print("❌ ERREUR PENDANT L'ENTRAÎNEMENT!")
    print("   Vérifiez les logs ci-dessus pour les détails")


🚀 DÉMARRAGE DE L'ENTRAÎNEMENT FINE-TUNING COMPLET
🚀 DÉMARRAGE DE L'ENTRAÎNEMENT FINE-TUNING

📊 Création des générateurs de données...
🔍 Filtrage des recettes avec images valides...


Validation images: 100%|██████████| 13463/13463 [00:15<00:00, 851.22it/s] 


✅ 13463/13463 images valides
📊 Split: 10774 train, 2689 validation
🔄 TripletGeneratorFT (Training):
   📊 Total recettes: 10774
   📸 Images: 10774
   🔄 Augmentation: ON (conservative)
🔍 Filtrage des recettes avec images valides...


Validation images: 100%|██████████| 13463/13463 [00:06<00:00, 1972.03it/s]


✅ 13463/13463 images valides
📊 Split: 10774 train, 2689 validation
🔄 TripletGeneratorFT (Validation):
   📊 Total recettes: 2689
   📸 Images: 2689
   🔄 Augmentation: OFF
🏗️ Création du modèle triplet fine-tuning...
✅ Modèle triplet créé:
   📐 Input shape: (None, 3, 224, 224, 3)
   📐 Output shape: (None, 3, 512)

⚡ Compilation du modèle avec learning rates optimisés...
   📈 Optimizer: Adam (lr=5e-05, wd=0.0001)
   🎯 Triplet Loss margin: 0.2
   📊 Métriques: accuracy, pos_sim, neg_sim

📋 Configuration des callbacks...
   💾 Model checkpoint: ./ft/best_recipe_image_retrieval_model_ft.keras
   ⏰ Early stopping: patience=5
   📉 Reduce LR: patience=3, factor=0.5
   🔻 Min LR: 1e-07

🎯 Lancement de l'entraînement (25 époques max)...
Epoch 1/25
168/673 ━━━━━━━━━━━━━━━━━━━━ 20:57 2s/step - average_negative_similarity: 5.1980e-04 - average_positive_similarity: 0.2415 - loss: 0.0272 - triplet_accuracy: 0.9753

KeyboardInterrupt: 

In [ ]:
# 🔧 INITIALISATION DU SYSTÈME DE RÉCUPÉRATION FINE-TUNING
# ============================================================================

print("🔧 Initialisation du système de récupération fine-tuning...")

# Charger le modèle entraîné
model_path_ft = f"ft/{CONFIG_FT['MODEL_NAME']}"

if os.path.exists(model_path_ft):
    # Créer le système de récupération
    retrieval_system_ft = RecipeImageRetrievalFT(
        model_path=model_path_ft,
        recipes_df=recipes_with_images.copy()
    )
    
    print(f"\n🔨 Construction de la base de données d'embeddings...")
    retrieval_system_ft.build_database(force_rebuild=False)
    
    print(f"\n✅ SYSTÈME DE RÉCUPÉRATION FINE-TUNING PRÊT!")
    print(f"   🏗️ Modèle: {model_path_ft}")
    print(f"   📊 Base d'embeddings: {retrieval_system_ft.embeddings_db.shape if retrieval_system_ft.embeddings_db is not None else 'Non créée'}")
    print(f"   📋 Recettes dans la base: {len(retrieval_system_ft.recipes_df) if retrieval_system_ft.recipes_df is not None else 0}")
    
else:
    print(f"❌ Modèle non trouvé: {model_path_ft}")
    print("   Vous devez d'abord entraîner le modèle!")
    retrieval_system_ft = None


In [ ]:
# 🎲 TEST AVEC IMAGES ALÉATOIRES - FINE-TUNING
# ============================================================================

if retrieval_system_ft is not None:
    print("🎲 Test avec des images aléatoires du dataset...")
    
    # Sélectionner quelques recettes aléatoires pour les tests
    test_recipes = retrieval_system_ft.recipes_df.sample(n=3, random_state=42)
    
    for i, (_, recipe) in enumerate(test_recipes.iterrows()):
        print(f"\n{'='*60}")
        print(f"🧪 TEST {i+1}/3: {recipe['Title']}")
        print(f"{'='*60}")
        
        try:
            # Recherche
            results = retrieval_system_ft.search_similar_recipes(
                query_image_path=recipe['image_path'],
                top_k=5,
                min_similarity=0.3
            )
            
            if results:
                print(f"✅ {len(results)} résultats trouvés!")
                
                # Afficher les résultats
                print(f"\n📊 TOP RÉSULTATS:")
                for j, result in enumerate(results[:3]):
                    print(f"  #{j+1}. {result['recipe_title'][:50]}...")
                    print(f"      📊 Similarité: {result['similarity']:.4f}")
                
                # Vérifier si la même recette est en première position
                if len(results) > 0 and results[0]['recipe_title'] == recipe['Title']:
                    print(f"🎯 PARFAIT! Même recette trouvée en premier (sim: {results[0]['similarity']:.4f})")
                else:
                    print(f"⚠️ Recette différente en premier: {results[0]['recipe_title'][:40]}")
                
            else:
                print("❌ Aucun résultat trouvé (similarité trop faible)")
                
        except Exception as e:
            print(f"❌ Erreur lors du test: {e}")
    
    print(f"\n{'='*60}")
    print("✅ TESTS AVEC IMAGES ALÉATOIRES TERMINÉS")
    print(f"{'='*60}")
    
else:
    print("❌ Système de récupération non initialisé!")
    print("   Initialisez d'abord le système avec un modèle entraîné.")


In [ ]:
# 🖼️ TEST AVEC IMAGES CUSTOM - FINE-TUNING
# ============================================================================

if retrieval_system_ft is not None:
    print("🖼️ Test avec des images custom du dossier test_recipes...")
    
    # Dossier des images de test
    test_images_dir = "test_recipes"
    
    if os.path.exists(test_images_dir):
        # Lister toutes les images de test
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        test_images = []
        
        for ext in image_extensions:
            test_images.extend(glob.glob(os.path.join(test_images_dir, f"*{ext}")))
            test_images.extend(glob.glob(os.path.join(test_images_dir, f"*{ext.upper()}")))
        
        if test_images:
            print(f"📁 {len(test_images)} images trouvées dans {test_images_dir}")
            
            # Tester les 3 premières images (ou toutes si moins de 3)
            images_to_test = test_images[:min(3, len(test_images))]
            
            for i, test_image_path in enumerate(images_to_test):
                print(f"\n{'='*60}")
                print(f"🧪 TEST CUSTOM {i+1}/{len(images_to_test)}: {os.path.basename(test_image_path)}")
                print(f"{'='*60}")
                
                try:
                    # Recherche
                    results = retrieval_system_ft.search_similar_recipes(
                        query_image_path=test_image_path,
                        top_k=5,
                        min_similarity=0.2  # Seuil plus bas pour images externes
                    )
                    
                    if results:
                        print(f"✅ {len(results)} résultats trouvés!")
                        
                        # Afficher les détails
                        print(f"\n📊 RÉSULTATS DÉTAILLÉS:")
                        for j, result in enumerate(results):
                            print(f"  #{j+1}. {result['recipe_title']}")
                            print(f"      📊 Similarité: {result['similarity']:.4f}")
                            print(f"      📁 Path: {result['image_path']}")
                            print()
                        
                        # Évaluation qualitative
                        best_sim = results[0]['similarity']
                        if best_sim >= 0.7:
                            print(f"🌟 EXCELLENTE correspondance! (sim: {best_sim:.4f})")
                        elif best_sim >= 0.5:
                            print(f"👍 BONNE correspondance (sim: {best_sim:.4f})")
                        elif best_sim >= 0.3:
                            print(f"⚠️ Correspondance MOYENNE (sim: {best_sim:.4f})")
                        else:
                            print(f"❌ Correspondance FAIBLE (sim: {best_sim:.4f})")
                        
                    else:
                        print("❌ Aucun résultat trouvé (similarité trop faible)")
                        print("   Essayez de réduire min_similarity ou vérifiez l'image")
                        
                except Exception as e:
                    print(f"❌ Erreur lors du test: {e}")
                    import traceback
                    traceback.print_exc()
            
            print(f"\n{'='*60}")
            print("✅ TESTS AVEC IMAGES CUSTOM TERMINÉS")
            print(f"{'='*60}")
            
        else:
            print(f"❌ Aucune image trouvée dans {test_images_dir}")
            print(f"   Extensions supportées: {image_extensions}")
    else:
        print(f"❌ Dossier {test_images_dir} non trouvé!")
        print("   Créez le dossier et ajoutez-y des images de recettes pour les tests")
    
else:
    print("❌ Système de récupération non initialisé!")
    print("   Initialisez d'abord le système avec un modèle entraîné.")


In [ ]:
# 🎉 RÉSUMÉ FINAL - FINE-TUNING COMPLET
# ============================================================================

print("🎉 RÉSUMÉ FINAL DU NOTEBOOK FINE-TUNING")
print("=" * 80)

print(f"\n🔥 CONFIGURATION FINE-TUNING:")
print(f"   📐 Taille image: {CONFIG_FT['IMG_SIZE']}x{CONFIG_FT['IMG_SIZE']}")
print(f"   🧠 Dimension embedding: {CONFIG_FT['EMBEDDING_DIM']}")
print(f"   📚 Batch size: {CONFIG_FT['BATCH_SIZE']}")
print(f"   📈 Learning rate: {CONFIG_FT['FINETUNE_LR']}")
print(f"   🎯 Marge triplet: {CONFIG_FT['MARGIN']}")
print(f"   ❄️ Couches gelées: {CONFIG_FT['FREEZE_LAYERS']}")
print(f"   🔥 Couches dégelées: 20 dernières couches")

print(f"\n🏗️ ARCHITECTURE:")
print(f"   🖼️ Base: EfficientNetB0 pré-entraîné")
print(f"   🧠 Embedding: Couche dense {CONFIG_FT['EMBEDDING_DIM']}D + L2 norm")
print(f"   🎯 Loss: Triplet Loss avec hard negative mining")
print(f"   📊 Augmentation: Conservative pour fine-tuning")

print(f"\n📊 DONNÉES:")
if 'recipes_with_images' in globals():
    print(f"   📋 Recettes totales: {len(recipes_with_images)}")
    print(f"   📸 Images disponibles: {len(recipes_with_images)}")
    print(f"   🔄 Split: {int((1-CONFIG_FT['VALIDATION_SPLIT'])*100)}% train / {int(CONFIG_FT['VALIDATION_SPLIT']*100)}% validation")
else:
    print("   ❌ Données non chargées")

print(f"\n🤖 MODÈLE:")
model_path_ft = f"ft/{CONFIG_FT['MODEL_NAME']}"
if os.path.exists(model_path_ft):
    print(f"   ✅ Modèle entraîné: {model_path_ft}")
    print(f"   💾 Taille: {os.path.getsize(model_path_ft) / (1024*1024):.1f} MB")
else:
    print(f"   ❌ Modèle non trouvé: {model_path_ft}")

print(f"\n📊 SYSTÈME DE RÉCUPÉRATION:")
if 'retrieval_system_ft' in globals() and retrieval_system_ft is not None:
    print(f"   ✅ Système initialisé")
    if retrieval_system_ft.embeddings_db is not None:
        print(f"   📊 Base d'embeddings: {retrieval_system_ft.embeddings_db.shape}")
        print(f"   📋 Recettes indexées: {len(retrieval_system_ft.recipes_df)}")
    else:
        print(f"   ⚠️ Base d'embeddings non construite")
else:
    print(f"   ❌ Système non initialisé")

print(f"\n📈 PERFORMANCE ATTENDUE:")
print(f"   🎯 Objectif Accuracy: >95%")
print(f"   📈 Objectif Pos Similarity: >0.6")
print(f"   📉 Objectif Neg Similarity: <0.3")
print(f"   📏 Objectif Gap Pos-Neg: >0.3")

print(f"\n🧪 TESTS DISPONIBLES:")
print(f"   🎲 Images aléatoires du dataset")
print(f"   🖼️ Images custom du dossier test_recipes/")
print(f"   📊 Visualisation des résultats")

print(f"\n💡 UTILISATION:")
print(f"   1️⃣ Entraîner: train_fine_tuning()")
print(f"   2️⃣ Initialiser: RecipeImageRetrievalFT()")
print(f"   3️⃣ Rechercher: search_similar_recipes()")
print(f"   4️⃣ Visualiser: plot_training_results_ft()")

print(f"\n🔄 AMÉLIORATIONS PAR RAPPORT AU TRANSFER LEARNING:")
print(f"   🔥 Fine-tuning des 20 dernières couches")
print(f"   📈 Learning rate adapté (1e-5)")
print(f"   🛡️ Augmentation conservative")
print(f"   ⚖️ Weight decay pour régularisation")
print(f"   📊 Meilleure adaptation au domaine des recettes")

print(f"\n=" * 80)
print(f"🎯 NOTEBOOK FINE-TUNING COMPLET ET OPÉRATIONNEL!")
print(f"=" * 80)
